In [1]:
import numpy as np
import pandas as pd

# Модель для прогнозирования, например ARIMA
from sktime.forecasting.arima import ARIMA
# Визуализация временных рядов
from sktime.utils.plotting import plot_series
# Модули для кросс-валидации
from sktime.split import temporal_train_test_split, ExpandingWindowSplitter, SlidingWindowSplitter, SingleWindowSplitter # делит выборку на обучающие и тестовые
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError # Метрики ошибки (кросс-валидации) MSE, MAE, MAPE

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

In [3]:
y = np.log(web.DataReader(name='GDP', data_source='fred', start='1995-01-01'))
y.index = y.index.to_period(freq='M') # замена индекса на периодический. Иначе кросс-валидация не будет работать
# длина ряда
len(y)

120

In [4]:
# специфицируем модель для прогнозирования, например ARIMA(2,1,2) без сноса
forecaster = ARIMA(order=(1,0,1), trend='ct')

# разбиваем выбору на обучающую (первые 150) и тестовую
cv_strategy = SingleWindowSplitter(fh=np.arange(1, 11), window_length=110)

# инициализируем метрики
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError(), MeanAbsolutePercentageError()]

cv_res = evaluate(forecaster=forecaster, y=y, cv=cv_strategy, strategy="refit", return_data=False, scoring=metric) #evaluate - прогонка 
cv_res

,test_MeanSquaredError,test_MeanAbsoluteError,test_MeanAbsolutePercentageError,fit_time,pred_time,len_train_window,cutoff
0,0.00298,0.050684,0.004941,0.827489,0.021788,110,2022-04


In [ ]:
forecaster = ARIMA(order=(1,1,0), trend='c')

# разбиваем выбору на обучающую (первые 150) и тестовую
cv_strategy = SingleWindowSplitter(fh=np.arange(1, 11), window_length=110)

# инициализируем метрики
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError(), MeanAbsolutePercentageError()]

cv_res = evaluate(forecaster=forecaster, y=y, cv=cv_strategy, strategy="refit", return_data=False, scoring=metric) #evaluate - прогонка 
cv_res

### Валидация методом k-Fold (расширяем обучающую выборку)

In [ ]:
# специфицируем модель для прогнозирования, например ARIMA(2,1,2) без сноса
forecaster = ARIMA(order=(1,1,0), trend='ct')

# разбиваем выбору на обучающую (первые 150) и тестовую
cv_strategy = ExpandingWindowSplitter(fh=np.arange(1, 6), initial_window=80, step_length=1) # 6 на сколько прогноз, initial_window - обучающая выборка, step_length - на сколько 

# инициализируем метрики
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError(), MeanAbsolutePercentageError()]

cv_res = evaluate(forecaster=forecaster, y=y, cv=cv_strategy, strategy="refit", return_data=False, scoring=metric) #evaluate - прогонка 
cv_res

### Скользящая обучающая выборка

In [7]:
# специфицируем модель для прогнозирования, например ARIMA(2,1,2) без сноса
forecaster = ARIMA(order=(1,1,0), trend='ct')

# разбиваем выбору на обучающую (первые 150) и тестовую
cv_strategy = SlidingWindowSplitter(fh=np.arange(1, 6), initial_window=80, step_length=1)

# инициализируем метрики
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError(), MeanAbsolutePercentageError()]

cv_res = evaluate(forecaster=forecaster, y=y, cv=cv_strategy, strategy="refit", return_data=False, scoring=metric) #evaluate - прогонка 
cv_res

,test_MeanSquaredError,test_MeanAbsoluteError,test_MeanAbsolutePercentageError,fit_time,pred_time,len_train_window,cutoff
0,0.000020,0.003513,0.000358,0.096409,0.021603,80,2014-10
1,0.000140,0.009649,0.000982,0.055176,0.017447,10,2015-01
2,0.000307,0.016124,0.001640,0.093667,0.018564,10,2015-04
3,0.000052,0.006631,0.000674,0.061375,0.014871,10,2015-07
4,0.000167,0.010383,0.001053,0.094994,0.013160,10,2015-10
5,0.000545,0.020625,0.002091,0.168542,0.015286,10,2016-01
6,0.000809,0.024463,0.002477,0.075770,0.018360,10,2016-04
7,0.000401,0.016741,0.001693,0.073589,0.019384,10,2016-07
8,0.000047,0.004840,0.000488,0.100022,0.017720,10,2016-10
9,0.000095,0.007982,0.000805,0.102891,0.019106,10,2017-01


In [ ]:
# средняя MSE, MAE, MAPE
cv_res.iloc[:,:len(metric)].mean()

### Автоматизируем процесс для нескольких моделей, чтобы выбрать наилучшую

Проведём кросс-валидацию для сравнения нескольких моделей. Например, сравним

ARIMA(1,1,1) без сноса
ARIMA(1,1,1) со сносом
ARIMA(1,2,1) без сноса

In [ ]:
# Зададим список из специфицированных моделей прогнозирования (вместо ARIMA  можно использовать другие модели из sktime)
forecasters = [ARIMA(order=(1,0,1), trend='ct'), ARIMA(order=(1,1,0), trend='c'), ARIMA(order=(1,1,1), trend='n'), ARIMA(order=(1,2,0), trend='n')]

# специфицируем метод кросс-валидации. Например, SlidingWindowSplitter
cv_strategy = SlidingWindowSplitter(fh=np.arange(1, 6), initial_window=100, step_length=5)

# инициализируем метрики
metric = [MeanSquaredError(square_root=False), MeanAbsoluteError(), MeanAbsolutePercentageError()]

# датафрейм с метриками по столбцам
cv_data = pd.DataFrame(data=None, columns=['MSE', 'MAE', 'MAPE'])

for model in forecasters:
	print(model)
	cv_res = evaluate(forecaster=model, y=y, cv=cv_strategy, strategy="refit", return_data=False, scoring=metric)
	# print(df.iloc[:,:len(metric)].mean()) # метрики для каждой модели
	cv_data.loc[len(cv_data.index)] = cv_res.iloc[:,[0,1,2]].mean().values

# результаты кросс-валидации в виде датафрейма
cv_data

ARIMA(order=(1, 0, 1), trend='ct')
ARIMA(order=(1, 1, 0), trend='c')
ARIMA(order=(1, 1, 1), trend='n')
ARIMA(order=(1, 2, 0), trend='n')


,MSE,MAE,MAPE
0,0.004729,0.058121,0.005743
1,0.002349,0.039703,0.003927
2,0.006285,0.059303,0.005863
3,0.001056,0.020656,0.002056


In [ ]:
# Индекс наилучшей модели в зависимости от метрики
for i in range(cv_data.shape[1]):
	print(f'{cv_data.columns[i]}: model #={cv_data.iloc[:,i].argmin()}')

MSE: model #=3
MAE: model #=3
MAPE: model #=3
